In [2]:
import math
from itertools import product

In [3]:
def softmax_share(Vi, Vj, beta):
    return math.exp(beta * Vi) / (1 + math.exp(beta * Vi) + math.exp(beta * Vj))

def utility(Vi, Vj, rhoi, beta):
    return rhoi * softmax_share(Vi, Vj, beta)

In [ ]:
import numpy as np
from numba import njit
from tqdm import tqdm

@njit
def get_ratio_threshold(Vi, Vj, Vk, beta):
    """Calculates the threshold ratio r_j / r_i needed to flip the inequality."""
    ei = np.exp(beta * Vi)
    ej = np.exp(beta * Vj)
    ek = np.exp(beta * Vk)
    return (ei * (1 + ej + ek)) / (ej * (1 + ei + ek))

@njit
def find_valid_rho_pair(low_bound, high_bound, rho_vals):
    """Checks if any two values in rho_vals satisfy low_bound < r1/r0 < high_bound."""
    # Since rho_vals is sorted, we can use a two-pointer approach or simple search
    for i in range(len(rho_vals)):
        r_low = rho_vals[i]
        for j in range(i + 1, len(rho_vals)):
            ratio = rho_vals[j] / r_low
            if low_bound < ratio < high_bound:
                return rho_vals[i], rho_vals[j]
            if ratio >= high_bound:
                break
    return -1.0, -1.0

@njit
def search_cycle(V_vals, rho_vals, beta):
    for V10 in V_vals:
        for V11 in V_vals:
            for V20 in V_vals:
                for V21 in V_vals:
                    # Condition for r1: 
                    # U11_00 > U10_00 => r11/r10 > Ratio(V10, V11, V20)
                    # U10_11 > U11_11 => r11/r10 < Ratio(V10, V11, V21)
                    L1 = get_ratio_threshold(V10, V11, V20, beta)
                    H1 = get_ratio_threshold(V10, V11, V21, beta)
                    
                    if L1 < H1:
                        r10, r11 = find_valid_rho_pair(L1, H1, rho_vals)
                        if r10 != -1.0:
                            # Condition for r2:
                            # U21_00 > U20_00 => r21/r20 > Ratio(V20, V21, V10)
                            # U20_11 > U21_11 => r21/r20 < Ratio(V20, V21, V11)
                            L2 = get_ratio_threshold(V20, V21, V10, beta)
                            H2 = get_ratio_threshold(V20, V21, V11, beta)
                            
                            if L2 < H2:
                                r20, r21 = find_valid_rho_pair(L2, H2, rho_vals)
                                if r20 != -1.0:
                                    return (V10, V11, V20, V21, r10, r11, r20, r21)
    return None

# Setup Grids
V_vals = np.arange(0.0, 20, 0.1)
rho_vals = np.arange(0.1, 20, 0.1)
beta = 1.0

print("Searching for cycle...")
result = search_cycle(V_vals, rho_vals, beta)

if result:
    V10, V11, V20, V21, r10, r11, r20, r21 = result
    print("CYCLE FOUND")
    print(f"beta={beta}")
    print(f"V1=(0:{V10}, 1:{V11}), rho1=(0:{r10}, 1:{r11})")
    print(f"V2=(0:{V20}, 1:{V21}), rho2=(0:{r20}, 1:{r21})")
else:
    print("No cycle found.")

Searching for cycle...
